## Importación librerías

In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

## Cargar Dataset y visualizar las primeras 5 filas

In [2]:
db = pd.read_csv("train.csv")
print("Dimensionalidad:",db.shape)
db.head()

Dimensionalidad: (891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Tratamiento previo del Dataset
### Tratamiento de variables que no aportan información
Primero elimino la columna PassengerId ya que no aporta información relevante para el análisis (es un identificador único de cada pasajero).


In [3]:
# Eliminar columna PassengerId
db.drop("PassengerId",axis=1,inplace=True)

Usaré de índice de la tabla la columna Name, ya que es un identificador de cada pasajero, pero a diferencia de "PassengerId", nos puede aportar información entre pasajeros. Para usar la columna como índice, primero vamos a comprobar si hay duplicados en la columna "Name", para que no haya filas con el mismo índice index.

En el caso de que no haya duplicados, usaremos la columna como índice del DataFrame.

In [4]:
# Calcular % de duplicados en Name
print("Duplicados en 'Name':",db["Name"].duplicated().sum()/db.shape[0]*100,"%")

Duplicados en 'Name': 0.0 %


In [5]:
# Utilizar columna Name como índice
db.set_index("Name",inplace=True)

### Tratamiento de valores nulos




In [6]:
# Calcular % de datos nulos por columna
for col in db.columns:
    print(col,":",db[col].isna().sum()/db.shape[0]*100,"%")

Survived : 0.0 %
Pclass : 0.0 %
Sex : 0.0 %
Age : 19.865319865319865 %
SibSp : 0.0 %
Parch : 0.0 %
Ticket : 0.0 %
Fare : 0.0 %
Cabin : 77.10437710437711 %
Embarked : 0.22446689113355783 %


#### Tratamiento de valores nulos en la columna "Cabin"

Primeramente la columna Cabin, que tiene un 77% de valores nulos. Esta columna la eliminaré porque más de 3/4 del dataset tienen esta columna incompleta.

In [7]:
# Eliminar columna Cabin
db.drop("Cabin",axis=1,inplace=True)

#### Tratamiento de valores nulos en la columna "Age"
Para la columna "Age" tenemos un 19% de valores nulos, por lo que rellenaré estos valores con algún método.

El método que voy a usar es el de imputación iterativa MICE (Multiple Imputation by Chained Equations). Este método imputa valores a partir de los valores de otras columnas.

Dejaremos este paso para el final ya que tendremos que tener un dataset limpio y de únicamente variables numéricas para poder aplicar este método

#### Tratar valores nulos en la columna "Embarked"
Al ver que el % de la columna "Embarked" es muy bajo, voy a imprimir los caso aislados para ver como puedo tratar estos elementos nulos.

In [8]:
# Imprimir casos nulos de Embarked
db[db["Embarked"].isna()]

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
Name,,,,,,,,,
"Icard, Miss. Amelie",1,1,female,38.0,0,0,113572,80.0,NaN
"Stone, Mrs. George Nelson (Martha Evelyn)",1,1,female,62.0,0,0,113572,80.0,NaN


Al ser dos casos aislados, decidí buscar información sobre estos pasajeros del Titanic para ver si podía rellenar los valores. Encontré las siguientes fuentes:

[Icard, Miss. Amelie](https://www.encyclopedia-titanica.org/titanic-survivor/amelia-icard.html)

[Stone, Mrs. George Nelson (Martha Evelyn)](https://www.encyclopedia-titanica.org/titanic-survivor/martha-evelyn-stone.html)

En este caso las dos mujeres embarcarón en Southampton, por lo que rellenaré los valores nulos de la columna "Embarked" con "S".

> **CURIOSIDAD:** Amelie Icard era la criada de Martha Evelyn por lo que tienen el mismo número de Ticket.

In [9]:
#Rellanamos los valores nulos de Embarked con S de Southampton
db["Embarked"].fillna("S",inplace=True)

### Tratamiento de variables duplicadas
Comprobamos si hay duplicados en las columnas que no nos interese que haya (en este caso, la única columna es Ticket).

In [10]:
# Calculamos % de duplicados en ticket
print("Duplicados en 'Ticket':",db["Ticket"].duplicated().sum()/db.shape[0]*100,"%")

Duplicados en 'Ticket': 23.56902356902357 %


Hay un 24% de duplicados en la columna, pero al ver anteriormente el caso donde usaban el mismo Ticket diferentes personas, decidí eliminar la columna (en el caso anterior usaban el mismo Ticket una mujer y su criada).

Ejemplo de Ticket duplicado (en este caso, la familia usaba un mismo Ticket):

In [11]:
# Mostrar filas ejemplo de Ticket duplicado
db[db["Ticket"]=="CA. 2343"]


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
Name,,,,,,,,,
"Sage, Master. Thomas Henry",0,3,male,NaN,8,2,CA. 2343,69.55,S
"Sage, Miss. Constance Gladys",0,3,female,NaN,8,2,CA. 2343,69.55,S
"Sage, Mr. Frederick",0,3,male,NaN,8,2,CA. 2343,69.55,S
"Sage, Mr. George John Jr",0,3,male,NaN,8,2,CA. 2343,69.55,S
"Sage, Miss. Stella Anna",0,3,female,NaN,8,2,CA. 2343,69.55,S
"Sage, Mr. Douglas Bullen",0,3,male,NaN,8,2,CA. 2343,69.55,S
"Sage, Miss. Dorothy Edith ""Dolly""",0,3,female,NaN,8,2,CA. 2343,69.55,S


In [12]:
# Eliminar columna Ticket
db.drop("Ticket",axis=1,inplace=True)

#### Tratamiento de valores nulos en la columna "Age"

Primero habrá que pasar las columnas de categóricas a numéricas para poder aplicar el método MICE.

In [13]:
# Mapear columnas Embarked y Sex
db["Embarked"] = db["Embarked"].map({"S": 0, "C": 1, "Q": 2})
db["Sex"] = db["Sex"].map({"male": 0, "female": 1})

db.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Name,,,,,,,,
"Braund, Mr. Owen Harris",0,3,0,22.0,1,0,7.2500,0
"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",1,1,1,38.0,1,0,71.2833,1
"Heikkinen, Miss. Laina",1,3,1,26.0,0,0,7.9250,0
"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,1,35.0,1,0,53.1000,0
"Allen, Mr. William Henry",0,3,0,35.0,0,0,8.0500,0


##### Método MICE (en Python)
Para el método MICE en Python usaremos la librería [sklearn.impute](https://scikit-learn.org/stable/modules/impute.html#iterative-imputer).

Para entender un poco en que consiste este método por encima, me he informado en la siguiente [web](https://www.numpyninja.com/post/mice-algorithm-to-impute-missing-values-in-a-dataset)

In [14]:
# Crear una instancia del imputador MICE
mice_imputer = IterativeImputer(initial_strategy="mean", estimator=LinearRegression())

# Imputar los valores faltantes
db = pd.DataFrame(mice_imputer.fit_transform(db), columns=db.columns,index=db.index)

'''
Limitar los valores imputados al rango de 0 a 100, ya que me daba valores negativos en caso de no limitar el rango.
'''
db["Age"] = np.clip(db["Age"], 0, 100)

# Convertir los valores imputados a enteros
db["Age"] = db["Age"].astype(int)

Finalmente comprobamos que no hay mas valores nulos en el dataset y exportamos el dataset limpio a un csv.

In [15]:
# Calcular % de datos nulos por columna
for col in db.columns:
    print(col,":",db[col].isnull().sum()/db.shape[0]*100,"%")

# Exportar dataset limpio a csv
db.to_csv("train_clean.csv")

Survived : 0.0 %
Pclass : 0.0 %
Sex : 0.0 %
Age : 0.0 %
SibSp : 0.0 %
Parch : 0.0 %
Fare : 0.0 %
Embarked : 0.0 %
